In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import Input, Model, Sequential
import keras
import time
#from keras.utils import np_utils
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import LSTM, RNN, Dense, Bidirectional, Input,Dropout,BatchNormalization,Flatten,\
                Activation, Conv3D, Concatenate, Reshape, Conv2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,\
        precision_score, recall_score, make_scorer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasClassifier
#from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
import concurrent.futures
import collections
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import AdamW, Adam
import random

2023-09-15 15:35:14.791861: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/')

In [3]:
data = pd.read_csv("words_250000_train.txt", header = None).rename(columns = {0:'word'}).dropna().reset_index(drop=True)

In [5]:
data.shape

(227299, 1)

In [4]:
data_train, data_test = train_test_split(data, test_size=0.4, random_state=42)

In [5]:
data_train.shape, data_test.shape

((136379, 1), (90920, 1))

<a name="split"></a>
## Partition the words into training and validation sets

In [6]:
print('Training with {} words'.format(data_train.shape[0]))
print('Testing with {} words'.format(data_test.shape[0]))

MAX_NUM_INPUTS = max([len(i) for i in data_train.word.values])
NUM_EPOCHS = 100
print('Max word length on Training Set: {}'.format(MAX_NUM_INPUTS))

Training with 136379 words
Testing with 90920 words
Max word length on Training Set: 28


In [7]:
FULL_DICTIONARY = data_train.word.to_list()
len(FULL_DICTIONARY)

136379

In [8]:
def build_ngram_models(dictionary):

  # create a nested dictionary that stores the occurrences of letter sequences ranging from 1 to 5 characters in length.
  # the nested dictionary will have an additional level to account for the length of each word in unigrams and bigrams.
  # for the unigram level, consider only the unique letters within each word.

  unigram = collections.defaultdict(lambda: collections.defaultdict(int))
  bigram = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))
  trigram = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))
  fourgram = collections.defaultdict(lambda:collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int))))
  fivegram = collections.defaultdict(lambda: collections.defaultdict(lambda:collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))))

  # iterating through each word in the dictionary
  # count the occurrences of letter sequences in words from the dictionary and update the n-gram models accordingly.
  for word in dictionary:
      # check each letter in the dictionary and update the ngram
      for i in range(len(word) - 4):
          # We exclude the last four letters of the word because it is searching for patterns of
          # four consecutive letters with a blank in the fifth position. Since the last four letters
          # cannot form such a pattern, there is no need to check them, resulting in improved efficiency
          # and focusing on the relevant parts of the word.

          bigram[len(word)][word[i]][word[i+1]] += 1
          trigram[word[i]][word[i+1]][word[i+2]] += 1
          fourgram[word[i]][word[i+1]][word[i+2]][word[i+3]] += 1
          fivegram[word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]] += 1

      i = len(word) - 4

      # fill rest of the ngrams for words very small words and complete coverage
      if len(word) == 2:
          bigram[len(word)][word[0]][word[1]] += 1
      elif len(word) == 3:
          bigram[len(word)][word[0]][word[1]] += 1
          bigram[len(word)][word[1]][word[2]] += 1
          trigram[word[0]][word[1]][word[2]] += 1
      # fill out rest of the fourgrams
      elif len(word) >= 4:
          bigram[len(word)][word[i]][word[i+1]] += 1
          bigram[len(word)][word[i+1]][word[i+2]] += 1
          bigram[len(word)][word[i+2]][word[i+3]] += 1
          trigram[word[i]][word[i+1]][word[i+2]] += 1
          trigram[word[i+1]][word[i+2]][word[i+3]] += 1
          fourgram[word[i]][word[i+1]][word[i+2]][word[i+3]] += 1

      # fill out unigrams
      for letter in set(word):
          unigram[len(word)][letter] += 1

  return unigram, bigram, trigram, fourgram, fivegram

In [9]:
UNIGRAM, BIGRAM, TRIGRAM, FOURGRAM, FIVEGRAM = build_ngram_models(FULL_DICTIONARY)

In [8]:
len(sorted(set("".join(FULL_DICTIONARY))))

26

In [10]:
def call_method(model, input, training = True):
  return model(input, training = training)

In [36]:
class HangmanPlayer:
    def __init__(self, word, model, lives=6):
        self.original_word = word
        self.full_word = [ord(i)-97 for i in word]
        self.letters_guessed = set([])
        self.letters_remaining = set(self.full_word)
        self.letters_remaining_count = collections.Counter(self.full_word)
        self.lives_left = lives
        self.obscured_words_seen = []
        self.letters_previously_guessed = []
        self.correct_responses = []
        self.z = model
        self.guessed_letters = []
        self.full_dictionary = FULL_DICTIONARY
        self.letter_set = sorted(set("".join(self.full_dictionary)))
        self.unigram_probabilities = [0] * len(self.letter_set)
        self.bigram_probabilities = [0] * len(self.letter_set)
        self.trigram_probabilities = [0] * len(self.letter_set)
        self.fourgram_probabilities = [0] * len(self.letter_set)
        self.fivegram_probabilities = [0] * len(self.letter_set)
        self.unigram, self.bigram, self.trigram, self.fourgram, self.fivegram = UNIGRAM, BIGRAM, TRIGRAM, FOURGRAM, FIVEGRAM
        self.encoded_n_grams_current_fivegram_probabilities = []
        self.encoded_n_grams_current_fourgram_probabilities = []
        self.encoded_n_grams_current_trigram_probabilities = []
        self.encoded_n_grams_current_bigram_probabilities = []
        self.encoded_n_grams_current_unigram_probabilities = []
        return


    def fivegram_probability(self, word):

      #given an input word in a clean format with no spaces and placeholders ('_') for unknown letters,
      #the process utilizes tri-grams to determine the likelihood of a specific letter appearing in a five-letter sequence for a word of a given length.
      #the output provides the probabilities for each letter, which will be utilized in the subsequent stage.

      # vector of probabilities for each letter
        probs = [0] * len(self.letter_set)

        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find patterns that have three consecutive letters where one of them is blank
        for i in range(len(word) - 4):
          # We exclude the last four letters of the word because it is searching for patterns of
          # four consecutive letters with a blank in the fifth position. Since the last four letters
          # cannot form such a pattern, there is no need to check them, resulting in improved efficiency
          # and focusing on the relevant parts of the word.

            # case 1: "eg word:  xyz_ "
            if word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] == '_':
                anchor_letter1 = word[i]
                anchor_letter2 = word[i+1]
                anchor_letter3 = word[i+2]
                anchor_letter4 = word[i+3]

                # calculate occurences of "anchor_letter1 anchor_letter2 blank" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[anchor_letter1][anchor_letter2][anchor_letter3][anchor_letter4][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[anchor_letter1][anchor_letter2][anchor_letter3][anchor_letter4][letter]
                        letter_count[j] += self.fivegram[anchor_letter1][anchor_letter2][anchor_letter3][anchor_letter4][letter]

            # case 2: "eg word: xyz_w "
            elif word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] == '_' and word[i+4] != '_':
                anchor_letter1 = word[i]
                anchor_letter2 = word[i+1]
                anchor_letter3 = word[i+2]
                anchor_letter4 = word[i+4]

                # calculate occurences of "anchor_letter1 blank anchor_letter2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[anchor_letter1][anchor_letter2][anchor_letter3][letter][anchor_letter4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[anchor_letter1][anchor_letter2][anchor_letter3][letter][anchor_letter4]
                        letter_count[j] += self.fivegram[anchor_letter1][anchor_letter2][anchor_letter3][letter][anchor_letter4]

            # case 3: "eg word: wx_yz "
            elif word[i] != '_' and word[i+1] != '_' and word[i+2] == '_' and word[i+3] != '_' and word[i+4] != '_':
                anchor_letter1 = word[i]
                anchor_letter2 = word[i+1]
                anchor_letter3 = word[i+3]
                anchor_letter4 = word[i+4]

                # calculate occurences of "blank anchor_letter1 anchor_letter2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[anchor_letter1][anchor_letter2][letter][anchor_letter3][anchor_letter4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[anchor_letter1][anchor_letter2][letter][anchor_letter3][anchor_letter4]
                        letter_count[j] += self.fivegram[anchor_letter1][anchor_letter2][letter][anchor_letter3][anchor_letter4]

            # case 4: "eg word: x_wyz"
            elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] != '_':
                anchor_letter1 = word[i]
                anchor_letter2 = word[i+2]
                anchor_letter3 = word[i+3]
                anchor_letter4 = word[i+4]

                # calculate occurences of "blank anchor_letter1 anchor_letter2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[anchor_letter1][letter][anchor_letter2][anchor_letter3][anchor_letter4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[anchor_letter1][letter][anchor_letter2][anchor_letter3][anchor_letter4]
                        letter_count[j] += self.fivegram[anchor_letter1][letter][anchor_letter2][anchor_letter3][anchor_letter4]

            # case 5: "eg word: _xwyz"
            elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] != '_':
                anchor_letter1 = word[i+1]
                anchor_letter2 = word[i+2]
                anchor_letter3 = word[i+3]
                anchor_letter4 = word[i+4]

                # calculate occurences of "blank anchor_letter1 anchor_letter2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[letter][anchor_letter1][anchor_letter2][anchor_letter3][anchor_letter4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[letter][anchor_letter1][anchor_letter2][anchor_letter3][anchor_letter4]
                        letter_count[j] += self.fivegram[letter][anchor_letter1][anchor_letter2][anchor_letter3][anchor_letter4]

        # calculate the probabilities of each letter
        if total_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        self.fivegram_probabilities = probs
        # interpolate probabilities between trigram and bigram
        """
       The step of multiplying each probability in probs by 0.40 and adding it to
       the corresponding probability in self.probabilities depicts interpolation.
       It is performed to combine the probabilities obtained from the fivegram level with the
       existing probabilities from the previous levels (trigram and bigram).This interpolation
       helps to balance the influence of higher-level ngrams (trigrams and bigrams) with the
       more specific information provided by the fivegram model.The method assigns a lower weight
       to the probabilities derived from the fivegram model. The factor of 0.40 determines the
       weight assigned to the fivegram probabilities, while the remaining weight (0.60) is assigned
       to the existing probabilities in self.probabilities. Overall, the interpolation step helps in
       combining the information from different ngram models to make more accurate predictions about
       the likelihood of specific letters appearing in the target blank space, considering both local
       and global patterns in the word.
        """
        '''
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + probs[i] * (0.40)
        '''


    def fourgram_probability(self, word):

      # given a word in a clean format without spaces and placeholders ('_') for unknown letters,
      # the process utilizes tri-grams to determine the probabilities of specific letters appearing in a four-letter sequence for a word of a given length.
      # the output provides the probabilities for each letter, which will be utilized in the next stage.


        # vector of probabilities for each letter
        probs = [0] * len(self.letter_set)

        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # calculates the probabilities of each letter in a word based on its context using a four-gram model.
        # It considers different cases based on the positions of underscores (_) in the word and updates the letter probabilities accordingly.
        # The probabilities are then interpolated with the existing probabilities from lower-level n-gram models (trigram and bigram)
        # to balance the influence of higher-level n-grams. The function then proceeds to the next level of the n-gram model to further
        # calculate the probabilities.

        # traverse the word and find patterns that have three consecutive letters where one of them is blank
        for i in range(len(word) - 3):

            # case 1: "eg word: abc_"
            if word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] == '_':
                anchor_letter1 = word[i]
                anchor_letter2 = word[i+1]
                anchor_letter3 = word[i+2]

                # calculate occurences of "anchor_letter1 anchor_letter2 blank" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[anchor_letter1][anchor_letter2][anchor_letter3][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[anchor_letter1][anchor_letter2][anchor_letter3][letter]
                        letter_count[j] += self.fourgram[anchor_letter1][anchor_letter2][anchor_letter3][letter]

            # case 2:  "eg word: ab_c"
            elif word[i] != '_' and word[i+1] != '_' and word[i+2] == '_' and word[i+3] != '_':
                anchor_letter1 = word[i]
                anchor_letter2 = word[i+1]
                anchor_letter3 = word[i+3]

                # calculate occurences of "anchor_letter1 blank anchor_letter2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[anchor_letter1][anchor_letter2][letter][anchor_letter3] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[anchor_letter1][anchor_letter2][letter][anchor_letter3]
                        letter_count[j] += self.fourgram[anchor_letter1][anchor_letter2][letter][anchor_letter3]

            # case 3: "eg word: a_bc"
            elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_' and word[i+3] != '_':
                anchor_letter1 = word[i]
                anchor_letter2 = word[i+2]
                anchor_letter3 = word[i+3]

                # calculate occurences of "blank anchor_letter1 anchor_letter2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[anchor_letter1][letter][anchor_letter2][anchor_letter3] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[anchor_letter1][letter][anchor_letter2][anchor_letter3]
                        letter_count[j] += self.fourgram[anchor_letter1][letter][anchor_letter2][anchor_letter3]

            # case 4:  "eg word: _abc"
            elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_':
                anchor_letter1 = word[i+1]
                anchor_letter2 = word[i+2]
                anchor_letter3 = word[i+3]

                # calculate occurences of "blank anchor_letter1 anchor_letter2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[letter][anchor_letter1][anchor_letter2][anchor_letter3] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[letter][anchor_letter1][anchor_letter2][anchor_letter3]
                        letter_count[j] += self.fourgram[letter][anchor_letter1][anchor_letter2][anchor_letter3]

        # calculate the probabilities of each letter
        if total_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        self.fourgram_probabilities = probs

        '''
        # interpolate probabilities between trigram and bigram

        """
        Multiply each probability in probs by 0.25 and add it to the corresponding probability in self.probabilities.
        This interpolation step combines the probabilities obtained from the fourgram model with the existing
        probabilities from the previous levels (trigram and bigram). It balances the influence of higher-level
        ngrams with the more specific information provided by the fourgram model.
        """
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + probs[i] * (0.25)
        '''


    def trigram_probability(self, word):

      # given a word in a clean format without spaces and placeholders ('_') for unknown letters,
      # the process utilizes tri-grams to determine the probabilities of specific letters appearing in a three-letter sequence for a word of a given length.
      # the output provides the probabilities for each letter, which will be utilized in the next stage.

        # vector of probabilities for each letter
        probs = [0] * len(self.letter_set)

        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find patterns that have three consecutive letters where one of them is blank
        for i in range(len(word) - 2):

            # case 1: "eg word: ab_"
            if word[i] != '_' and word[i+1] != '_' and word[i+2] == '_':
                anchor_letter1 = word[i]
                anchor_letter2 = word[i+1]

                # calculate occurences of "anchor_letter1 anchor_letter2 blank" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.trigram[anchor_letter1][anchor_letter2][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.trigram[anchor_letter1][anchor_letter2][letter]
                        letter_count[j] += self.trigram[anchor_letter1][anchor_letter2][letter]

            # case 2: "eg word: a_b"
            elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_':
                anchor_letter1 = word[i]
                anchor_letter2 = word[i+2]

                # calculate occurences of "anchor_letter1 blank anchor_letter2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.trigram[anchor_letter1][letter][anchor_letter2] > 0 and letter not in self.guessed_letters:
                        total_count += self.trigram[anchor_letter1][letter][anchor_letter2]
                        letter_count[j] += self.trigram[anchor_letter1][letter][anchor_letter2]

            # case 3: "eg word: _ab"
            elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_':
                anchor_letter1 = word[i+1]
                anchor_letter2 = word[i+2]

                # calculate occurences of "blank anchor_letter1 anchor_letter2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.trigram[letter][anchor_letter1][anchor_letter2] > 0 and letter not in self.guessed_letters:
                        total_count += self.trigram[letter][anchor_letter1][anchor_letter2]
                        letter_count[j] += self.trigram[letter][anchor_letter1][anchor_letter2]

        # calculate the probabilities of each letter
        if total_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count
        self.trigram_probabilities = probs

        '''
        # interpolate probabilities between trigram and bigram

        """
        Multiply each probability in probs by 0.20 and add it to the corresponding probability in self.probabilities.
        This interpolation step combines the probabilities obtained from the trigram model with the existing
        probabilities from the previous levels. It balances the influence of higher-level
        ngrams with the more specific information provided by the trigram model.
        """
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + probs[i] * (0.20)
        '''


    def bigram_probability(self, word):

      #given a word in a clean format without spaces and placeholders ('_') for unknown letters,
      #the process utilizes bi-grams to determine the probabilities of specific letters appearing in a two-letter sequence for a word of a given length.
      #these probabilities are then updated in the trigram_probability set.
      #the output provides the probabilities for each letter, which will be used in the next stage.

        # vector of probabilities for each letter
        probs = [0] * len(self.letter_set)

        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find either patterns of "letter blank" or "blank letter"
        for i in range(len(word) - 1):
            # case 1: "eg word: a_"
            if word[i] != '_' and word[i+1] == '_':
                anchor_letter = word[i]

                # calculate occurences of "anchor_letter blank" and each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.bigram[len(word)][anchor_letter][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.bigram[len(word)][anchor_letter][letter]
                        letter_count[j] += self.bigram[len(word)][anchor_letter][letter]

            # case 2: "eg word: _a"
            elif word[i] == '_' and word[i+1]!= '_':
                anchor_letter = word[i+1]

                # calculate occurences of "blank anchor_letter" and each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.bigram[len(word)][letter][anchor_letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.bigram[len(word)][letter][anchor_letter]
                        letter_count[j] += self.bigram[len(word)][letter][anchor_letter]

        # calculate the probabilities of each letter
        if total_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        self.bigram_probabilities = probs

        '''
        # interpolate probabilities between trigram and bigram
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + probs[i] * (0.10)
        '''


    def unigram_probability(self, word):

      # given a word in a clean format without spaces and placeholders ('_') for unknown letters,
      # the process utilizes unigrams to calculate the probabilities of specific letters appearing in any blank space.
      # These probabilities are then updated in the bigram_probability set.
      # The output provides the letter with the highest overall probability.

        # vector of probabilities for each letter
        probs = [0] * len(self.letter_set)

        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find blank spaces
        for i in range(len(word)):
            # case 1: "eg word: a_"
            if word[i] == '_':

                # calculate occurences of pattern and each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.unigram[len(word)][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.unigram[len(word)][letter]
                        letter_count[j] += self.unigram[len(word)][letter]

        # calculate the probabilities of each letter appearing
        if total_count > 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        self.unigram_probabilities = probs

        '''
        # interpolate probabilities
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + probs[i] * (0.05)

        # adjust probabilities so they sum to one
        final_probs = [0] * len(self.letter_set)
        if sum(self.probabilities) > 0:
            for i in range(len(self.probabilities)):
                final_probs[i] = self.probabilities[i] / sum(self.probabilities)

        self.probabilities = final_probs

        return

        '''
        '''
        # finding letter with highest probability
        max_prob = 0
        guess_letter = ''
        for i, letter in enumerate(self.letter_set):
            if self.probabilities[i] > max_prob:
                max_prob = self.probabilities[i]
                guess_letter = letter

        # if no letter chosen from above, pick a random one (extra weight on vowels)
        if guess_letter == '':
            letters = self.letter_set.copy()
            random.shuffle(letters)
            letters_shuffled = ['e','a','i','o','u'] + letters
            for letter in letters_shuffled:
                if letter not in self.guessed_letters:
                    return letter

        return guess_letter
        '''

    def encode_obscured_word(self):
        word = [i if i in self.letters_guessed else 26 for i in self.full_word]
        obscured_word = np.zeros((len(word), 27), dtype=np.float32)
        for i, j in enumerate(word):
            obscured_word[i, j] = 1
        return(obscured_word)

    def encode_obscured_word_n_grams(self):
        obscured_word = "".join([i if i in self.guessed_letters else "_" for i in self.original_word])
        return(obscured_word)

    def encode_previous_guesses(self):
        guess = np.zeros(26, dtype=np.float32)
        for i in self.letters_guessed:
            guess[i] = 1
        return(guess)

    def encode_correct_responses(self):
        response = np.zeros(26, dtype=np.float32)
        for i in self.letters_remaining:
            response[i] = self.letters_remaining_count[i]
        response /= response.sum()
        return(response)

    def store_guess_and_result_train(self, guess, letter_guessed, encoded_obscured_word, encoded_previous_guesses):
        self.obscured_words_seen.append(encoded_obscured_word)
        self.letters_previously_guessed.append(encoded_previous_guesses)
        self.encoded_n_grams_current_fivegram_probabilities.append(self.fivegram_probabilities)
        self.encoded_n_grams_current_fourgram_probabilities.append(self.fourgram_probabilities)
        self.encoded_n_grams_current_trigram_probabilities.append(self.trigram_probabilities)
        self.encoded_n_grams_current_bigram_probabilities.append(self.bigram_probabilities)
        self.encoded_n_grams_current_unigram_probabilities.append(self.unigram_probabilities)
        correct_responses = self.encode_correct_responses()
        self.correct_responses.append(correct_responses)
        self.letters_guessed.add(guess)
        self.guessed_letters.append(letter_guessed)
        if guess in self.letters_remaining:
            self.letters_remaining.remove(guess)
            del self.letters_remaining_count[guess]
        else:
            self.lives_left -= 1
        return

    def store_guess_and_result_test(self, guess, letter_guessed, encoded_obscured_word, encoded_previous_guesses):
        self.letters_guessed.add(guess)
        self.guessed_letters.append(letter_guessed)
        if guess in self.letters_remaining:
            self.letters_remaining.remove(guess)
            del self.letters_remaining_count[guess]
        else:
            self.lives_left -= 1
        return

    def run_train(self):
        while (self.lives_left > 0) and (len(self.letters_remaining) > 0):
              word_n_grams = self.encode_obscured_word_n_grams()
              self.fivegram_probability(word_n_grams)
              self.fourgram_probability(word_n_grams)
              self.trigram_probability(word_n_grams)
              self.bigram_probability(word_n_grams)
              self.unigram_probability(word_n_grams)
              encoded_obscured_word, encoded_previous_guesses = self.encode_obscured_word(), self.encode_previous_guesses()
              i = 1
              sorted_probs = np.squeeze(call_method(self.z, [tf.convert_to_tensor(np.array([encoded_obscured_word])),\
                                                            tf.convert_to_tensor(np.array([encoded_previous_guesses])),\
                                                            tf.convert_to_tensor(np.array([self.fivegram_probabilities])),\
                                                            tf.convert_to_tensor(np.array([self.fourgram_probabilities])),\
                                                            tf.convert_to_tensor(np.array([self.trigram_probabilities])),\
                                                            tf.convert_to_tensor(np.array([self.bigram_probabilities])),\
                                                            tf.convert_to_tensor(np.array([self.unigram_probabilities]))])).argsort()
              while sorted_probs[-i] in self.letters_guessed:
                i+= 1
              guess = sorted_probs[-i]
              letter_guessed = chr(guess+97)
              self.store_guess_and_result_train(guess, letter_guessed, encoded_obscured_word, encoded_previous_guesses)
        return(np.array(self.obscured_words_seen),
              np.array(self.letters_previously_guessed),
              np.array(self.encoded_n_grams_current_fivegram_probabilities),
              np.array(self.encoded_n_grams_current_fourgram_probabilities),
              np.array(self.encoded_n_grams_current_trigram_probabilities),
              np.array(self.encoded_n_grams_current_bigram_probabilities),
              np.array(self.encoded_n_grams_current_unigram_probabilities),
              np.array(self.correct_responses))


    def run_test(self):
        while (self.lives_left > 0) and (len(self.letters_remaining) > 0):
              word_n_grams = self.encode_obscured_word_n_grams()
              self.fivegram_probability(word_n_grams)
              self.fourgram_probability(word_n_grams)
              self.trigram_probability(word_n_grams)
              self.bigram_probability(word_n_grams)
              self.unigram_probability(word_n_grams)
              encoded_obscured_word, encoded_previous_guesses = self.encode_obscured_word(), self.encode_previous_guesses()
              i = 1
              sorted_probs = np.squeeze(call_method(self.z, [tf.convert_to_tensor(np.array([encoded_obscured_word])),\
                                                            tf.convert_to_tensor(np.array([encoded_previous_guesses])),\
                                                            tf.convert_to_tensor(np.array([self.fivegram_probabilities])),\
                                                            tf.convert_to_tensor(np.array([self.fourgram_probabilities])),\
                                                            tf.convert_to_tensor(np.array([self.trigram_probabilities])),\
                                                            tf.convert_to_tensor(np.array([self.bigram_probabilities])),\
                                                            tf.convert_to_tensor(np.array([self.unigram_probabilities]))], training = False)).argsort()
              while sorted_probs[-i] in self.letters_guessed:
                i+= 1
              guess = sorted_probs[-i]
              letter_guessed = chr(guess+97)
              self.store_guess_and_result_test(guess, letter_guessed, encoded_obscured_word, encoded_previous_guesses)
        return
    
    def evaluate_performance(self):
        # Assumes that the run() method has already been called
        ended_in_success = self.lives_left > 0
        letters_in_word = set([i for i in self.original_word])
        correct_guesses = len(letters_in_word) - len(self.letters_remaining)
        incorrect_guesses = len(self.letters_guessed) - correct_guesses
        return(ended_in_success, correct_guesses, incorrect_guesses, letters_in_word)

In [35]:
def hangman_wrapper(word, z):
  hangman_player = HangmanPlayer(word, z)
  words_seen, previous_letters, n_grams_guess, correct_responses = hangman_player.run_train()
  return words_seen, previous_letters, n_grams_guess, correct_responses

In [47]:
model_filename = 'hangman_model2.dnn'
from tensorflow.keras.models import load_model

# load model
z = load_model(model_filename)
# summarize model.
z.summary()

Model: "hangman_rl_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_obscured_word_seen (  [(None, None, 27)]           0         []                            
 InputLayer)                                                                                      
                                                                                                  
 lstm (LSTM)                 (None, 28)                   6272      ['input_obscured_word_seen[0][
                                                                    0]']                          
                                                                                                  
 input_letters_guessed_prev  [(None, 26)]                 0         []                            
 iously (InputLayer)                                                               

In [29]:
from tqdm import tqdm

def evaluation_wrapper(word, model):
  my_player = HangmanPlayer(word, model)
  my_player.run_test()
  return my_player.evaluate_performance()

In [ ]:
results = []
for idx, i in enumerate(data_test.word.to_list()):
    results.append(evaluation_wrapper(i, z))
result_df = pd.DataFrame(results, columns=['won', 'num_correct', 'num_incorrect', 'letters'])

In [48]:
eval_words = data_test.word.to_list()[:]
results = []
with tqdm(total=len(eval_words)) as pbar:
  with concurrent.futures.ThreadPoolExecutor(max_workers=500) as executor:
    future_to_hangman = [executor.submit(evaluation_wrapper, word, z) for word in eval_words]
    for future in concurrent.futures.as_completed(future_to_hangman):
        results.append(future.result())
        pbar.update(1)

result_df = pd.DataFrame(results, columns=['won', 'num_correct', 'num_incorrect', 'letters'])

100%|███████████████████████████████████| 90920/90920 [2:00:34<00:00, 12.57it/s]


Then we summarize the results:

In [49]:
print('Performance on the validation set:')
print('- Averaged {:0.5f} correct and {:0.5f} incorrect guesses per game'.format(result_df['num_correct'].mean(),
                                                                       result_df['num_incorrect'].mean()))
print('- Won {:0.5f}% of games played'.format(100 * result_df['won'].sum() / len(result_df.index)))

Performance on the validation set:
- Averaged 6.42811 correct and 4.41260 incorrect guesses per game
- Won 54.26859% of games played
